# 口罩识别

导入模块：

In [1]:
from pathlib import Path

from utils.file import mkdir, glob_file, gen_dataset, make_yaml

## 数据处理

In [2]:
root = Path(r'E:\kaggle\datasets\face-mask-detection')
annotations_path = root/'annotations'
images_path =  root/'images'

annotations_files = glob_file(root, '*.xml')
images_files = glob_file(root, '*.png')

In [3]:
annotations_files[:5], images_files[:5]

(['E:/kaggle/datasets/face-mask-detection/annotations/maksssksksss0.xml',
  'E:/kaggle/datasets/face-mask-detection/annotations/maksssksksss1.xml',
  'E:/kaggle/datasets/face-mask-detection/annotations/maksssksksss2.xml',
  'E:/kaggle/datasets/face-mask-detection/annotations/maksssksksss3.xml',
  'E:/kaggle/datasets/face-mask-detection/annotations/maksssksksss4.xml'],
 ['E:/kaggle/datasets/face-mask-detection/images/maksssksksss0.png',
  'E:/kaggle/datasets/face-mask-detection/images/maksssksksss1.png',
  'E:/kaggle/datasets/face-mask-detection/images/maksssksksss2.png',
  'E:/kaggle/datasets/face-mask-detection/images/maksssksksss3.png',
  'E:/kaggle/datasets/face-mask-detection/images/maksssksksss4.png'])

### 数据集制作

参考：https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data#3-organize-directories

In [25]:
config_dir = Path('datasets')
mkdir(config_dir)
labels = ['with_mask', 'mask_weared_incorrect', 'without_mask']
yaml_path = config_dir/'config.yaml'

In [26]:
from utils.file import split

In [27]:
def write(paths, save_path):
    with open(save_path, 'w') as fp:
        for path in paths:
            fp.write(path+'\n')

In [28]:
(images_train, labels_train), (images_val, labels_val), (images_test, labels_test) = \
        split(images_files, annotations_files)

In [31]:
write(images_train, config_dir/'train.txt')
write(images_val, config_dir/'val.txt')
write(images_test, config_dir/'test.txt')

make_yaml(config_dir.absolute().as_posix(), labels, yaml_path)

## 模型

In [32]:
import torch

model_name = 'yolov5l'
image_size = 640
batch_size = 4
epochs = 10
device = '1' if torch.cuda.is_available() else 'cpu'
saved_model_name = 'best.pt'

# for test
confidence_threshold = 0.25 # Threshold of object inference
iou_threshold = 0.45 # Threshold of remove overlapping boxes

device

'1'

## 训练

In [ ]:
# !wandb online

In [33]:
yolo_path = "E:/kaggle/yolov5/"

In [34]:
!python {yolo_path}train.py --weights {model_name}.pt \
        --data {yaml_path} \
        --epochs {epochs} --batch-size {batch_size} \
        --img-size {image_size} --device {device}

github: up to date with https://github.com.cnpmjs.org/daobook/yolov5 





train: weights=yolov5l.pt, cfg=, data=datasets\config.yaml, hyp=..\..\..\..\yolov5\data\hyps\hyp.scratch.yaml, epochs=10, batch_size=4, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=1, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=..\..\..\..\yolov5\runs\train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
YOLOv5  v6.0-189-gaf00134 torch 1.10.0 CUDA:1 (NVIDIA GeForce GTX 1080 Ti, 11264MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0.5, shear=

In [ ]:
saved_model = 'runs/train/exp/weights/' + saved_model_name
submission_path = 'submission/'
mkdir(submission_path)

In [ ]:
!mv {saved_model} {submission_path}{saved_model_name}
!rm -rf {pwd}runs/train/*
!ls {submission_path}

## 测试

In [ ]:
!python {yolo_path}detect.py --weights {submission_path}{saved_model_name} \
        --source {new_images_path + sub_directories[2]} --img-size {image_size} \
        --conf-thres {confidence_threshold} --iou-thres {iou_threshold} --device {device} \
        --hide-labels --hide-conf

In [ ]:
!mv runs/detect/exp/* {submission_path}
!rm -rf runs/detect/*
predict_images = [something for something in os.listdir(submission_path) if not os.path.isdir(submission_path + something) and something.endswith('.png')]
predict_images.sort(key = lambda e: int(re.sub('[^0-9]', '', e)))
predict_images